In [2]:
import pandas as pd
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, roc_curve
from sklearn.ensemble import RandomForestClassifier

Import data with the regions with already known oncogenes, get rid of unnecessary columns

In [ ]:
all_data = pd.read_csv("../../../Data/Processed_data/Selection_of_regions_for_ML.csv")

In [4]:
all_data.iloc[:, 3:]

,rank_expr_norm,rank_pubmed_mean_norm,rank_tf_norm,rank_kinase_norm,rank_crisp_mean_norm,rank_crisp_min_norm,rank_expr_fold_change_norm,rank_is_TSG_norm,rank_GO_term_norm,rank_PPI_norm,SUM_RANK,is_oncogene
0,0.50,0.60,0.60,0.60,0.40,0.40,0.50,0.60,0.60,0.60,5.40,0
1,0.50,0.20,0.60,0.60,1.00,1.00,0.80,0.60,0.20,0.60,6.10,0
2,1.00,0.80,0.60,0.60,0.60,0.80,1.00,0.60,1.00,0.60,7.60,0
3,0.50,1.00,0.60,0.60,0.20,0.20,0.50,0.60,0.40,0.60,5.20,0
4,0.50,0.40,0.60,0.60,0.80,0.60,0.20,0.60,0.80,0.60,5.70,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2291,0.19,0.38,0.53,0.53,0.12,0.12,0.12,0.53,0.47,0.53,3.52,0
2292,0.69,0.66,0.53,0.53,0.81,0.94,0.62,0.53,0.75,0.53,6.59,0
2293,0.94,0.88,0.53,0.53,1.00,0.75,0.25,0.53,0.84,0.53,6.78,0
2294,0.56,1.00,0.53,0.53,0.75,0.44,0.31,0.53,0.84,0.53,6.02,0


In [5]:
data = all_data.iloc[:, 3:]

Overall look of the table

In [6]:
data

,rank_expr_norm,rank_pubmed_mean_norm,rank_tf_norm,rank_kinase_norm,rank_crisp_mean_norm,rank_crisp_min_norm,rank_expr_fold_change_norm,rank_is_TSG_norm,rank_GO_term_norm,rank_PPI_norm,SUM_RANK,is_oncogene
0,0.50,0.60,0.60,0.60,0.40,0.40,0.50,0.60,0.60,0.60,5.40,0
1,0.50,0.20,0.60,0.60,1.00,1.00,0.80,0.60,0.20,0.60,6.10,0
2,1.00,0.80,0.60,0.60,0.60,0.80,1.00,0.60,1.00,0.60,7.60,0
3,0.50,1.00,0.60,0.60,0.20,0.20,0.50,0.60,0.40,0.60,5.20,0
4,0.50,0.40,0.60,0.60,0.80,0.60,0.20,0.60,0.80,0.60,5.70,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2291,0.19,0.38,0.53,0.53,0.12,0.12,0.12,0.53,0.47,0.53,3.52,0
2292,0.69,0.66,0.53,0.53,0.81,0.94,0.62,0.53,0.75,0.53,6.59,0
2293,0.94,0.88,0.53,0.53,1.00,0.75,0.25,0.53,0.84,0.53,6.78,0
2294,0.56,1.00,0.53,0.53,0.75,0.44,0.31,0.53,0.84,0.53,6.02,0


Target

In [7]:
all_data[["is_oncogene"]]

,is_oncogene
0,0
1,0
2,0
3,0
4,1
...,...
2291,0
2292,0
2293,0
2294,0


In [8]:
data.describe()

,rank_expr_norm,rank_pubmed_mean_norm,rank_tf_norm,rank_kinase_norm,rank_crisp_mean_norm,rank_crisp_min_norm,rank_expr_fold_change_norm,rank_is_TSG_norm,rank_GO_term_norm,rank_PPI_norm,SUM_RANK,is_oncogene
count,2296.000000,2296.000000,2296.000000,2296.000000,2296.000000,2296.000000,2296.000000,2296.000000,2296.000000,2296.000000,2296.000000,2296.000000
mean,0.547683,0.547687,0.548075,0.547770,0.547692,0.547696,0.547679,0.547748,0.547461,0.547443,5.476934,0.095383
std,0.290272,0.288908,0.134958,0.109563,0.290873,0.290901,0.290597,0.053923,0.284482,0.097097,1.217798,0.293808
min,0.020000,0.020000,0.120000,0.440000,0.010000,0.010000,0.010000,0.510000,0.040000,0.320000,2.320000,0.000000
25%,0.300000,0.290000,0.490000,0.500000,0.300000,0.300000,0.300000,0.520000,0.280000,0.510000,4.620000,0.000000
50%,0.550000,0.540000,0.500000,0.520000,0.550000,0.550000,0.545000,0.530000,0.530000,0.530000,5.460000,0.000000
75%,0.800000,0.800000,0.550000,0.550000,0.800000,0.800000,0.800000,0.560000,0.800000,0.560000,6.290000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000,1.000000


In [9]:
data.shape

(2296, 12)

Split data into train and test (consider using stratify)

In [12]:
my_y = data[['is_oncogene']].values.astype("int8")

In [13]:
my_x = data.drop('is_oncogene', axis=1).values

In [14]:
my_x

array([[0.5 , 0.6 , 0.6 , ..., 0.6 , 0.6 , 5.4 ],
       [0.5 , 0.2 , 0.6 , ..., 0.2 , 0.6 , 6.1 ],
       [1.  , 0.8 , 0.6 , ..., 1.  , 0.6 , 7.6 ],
       ...,
       [0.94, 0.88, 0.53, ..., 0.84, 0.53, 6.78],
       [0.56, 1.  , 0.53, ..., 0.84, 0.53, 6.02],
       [0.06, 0.81, 0.53, ..., 0.62, 0.53, 4.92]])

In [16]:
my_y = np.ravel(my_y)

In [17]:
import xgboost
import catboost

from sklearn.ensemble import (ExtraTreesClassifier, VotingClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score

In [18]:
SEED = 42
rf = RandomForestClassifier(n_estimators=300, random_state=SEED)
etc = ExtraTreesClassifier(n_estimators=300, random_state=SEED)
cat = catboost.CatBoostClassifier(verbose=0, random_seed=SEED)
xgb_rf = xgboost.XGBRFClassifier(random_state=SEED)
nb = GaussianNB()

In [19]:
base_models_new = [("RF", rf), ("ETC", etc), ("CAT", cat), ("XGB_RF", xgb_rf),
                   ("NB", nb)]

cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)
voting_soft = VotingClassifier(base_models_new, voting='soft')
score = cross_val_score(voting_soft, my_x, my_y, cv=cv, scoring="recall", n_jobs=-1)

In [20]:
score.mean()

0.5809206804462107

In [21]:
X_train, X_test, y_train, y_test = train_test_split(my_x, my_y, test_size=0.30, random_state=SEED) 

In [22]:
voting_soft.fit(X_train, y_train)

VotingClassifier(estimators=[('RF',
                              RandomForestClassifier(n_estimators=300,
                                                     random_state=42)),
                             ('ETC',
                              ExtraTreesClassifier(n_estimators=300,
                                                   random_state=42)),
                             ('CAT',
                              <catboost.core.CatBoostClassifier object at 0x000001A1996B9610>),
                             ('XGB_RF',
                              XGBRFClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bytree=None,
                                              device=Non...
                                              interaction_constraints=None,
                                              max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              objective='binary:logistic',
                                              random_state=42, reg_alpha=None, ...)),
                             ('NB', GaussianNB())],
                 voting='soft')

In [23]:
y_pred = voting_soft.predict(X_test)

In [24]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (how many objects got correct labels) is \n{accuracy}")
print()
precision = precision_score(y_test, y_pred)
print(f"Precision (how many oncogenes are actuall oncognes within predicted ones) is \n{precision}")
print()
recall = recall_score(y_test, y_pred)
print(f"Recall (how many oncogenes out of all oncogenes we got right) is \n{recall}")
print()
f1_score_result = f1_score(y_test, y_pred)
print(f"F1_score is \n{f1_score_result}")
print()
roc_auc = roc_auc_score(y_test, voting_soft.predict_proba(X_test)[:,1])
print(f"ROC-AUC (performance of model over multiple thresholds in our predicted probabilities) is \n{roc_auc}")

Accuracy (how many objects got correct labels) is 
0.9375907111756169

Precision (how many oncogenes are actuall oncognes within predicted ones) is 
0.75

Recall (how many oncogenes out of all oncogenes we got right) is 
0.5373134328358209

F1_score is 
0.6260869565217392

ROC-AUC (performance of model over multiple thresholds in our predicted probabilities) is 
0.9289005135096223


In [ ]:
new_data = pd.read_csv("../../../Data/Processed_data/Selection_of_regions_for_ML_without_known_oncognes.csv")

In [31]:
new_data

,Unnamed: 0,ID,gene_name,rank_expr_norm,rank_pubmed_mean_norm,rank_tf_norm,rank_kinase_norm,rank_crisp_mean_norm,rank_crisp_min_norm,rank_expr_fold_change_norm,rank_is_TSG_norm,rank_GO_term_norm,rank_PPI_norm,SUM_RANK,group_num
0,1,TCGA-02-2485_7_52654649_53869348,POM121L12,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,10.00,1
1,2,TCGA-02-2485_7_56084143_56403069,PSPH,0.33,1.00,0.58,0.50,0.50,0.67,0.33,0.58,1.00,0.58,6.07,2
2,3,TCGA-02-2485_7_56084143_56403069,CCT6A,0.83,0.83,0.58,0.50,1.00,1.00,1.00,0.58,0.83,0.58,7.73,2
3,4,TCGA-02-2485_7_56084143_56403069,SUMF2,0.67,0.33,0.58,0.50,0.33,0.33,0.50,0.58,0.25,0.58,4.65,2
4,5,TCGA-02-2485_7_56084143_56403069,PHKG1,0.50,0.50,0.58,1.00,0.67,0.50,0.67,0.58,0.67,0.58,6.25,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19098,19099,TCGA-ZG-A9M4_17_80369439_81049726,B3GNTL1,0.08,0.42,0.50,0.54,0.75,0.58,0.75,0.54,0.25,0.54,4.95,3344
19099,19100,TCGA-ZG-A9M4_17_80369439_81049726,METRNL,0.25,0.67,0.50,0.54,0.67,0.92,0.08,0.54,0.50,0.54,5.21,3344
19100,19101,TCGA-ZJ-AAXN_11_101110923_101584390,TRPC6,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,10.00,3345
19101,19102,TCGA-ZQ-A9CR_8_126360056_126797467,NSMCE2,0.50,0.50,0.75,0.50,1.00,1.00,0.50,0.75,0.50,0.75,6.75,3346


In [33]:
data_to_predict = new_data.iloc[:, 3:-1].values

In [34]:
my_pred_proba = voting_soft.predict_proba(data_to_predict)

In [35]:
first_values = my_pred_proba[:, 1]

In [39]:
new_data['is_onco_prediction_proba_ensemble'] = first_values

Export predicted oncogenes

In [43]:
gene_list_proba_089 = new_data[new_data['is_onco_prediction_proba_ensemble'] >= 0.89][['gene_name', 'is_onco_prediction_proba_ensemble']]

In [45]:
gene_list_proba_089.dropna(inplace = True)

In [ ]:
gene_list_proba_089.to_csv('../../../Data/Result_data/Prediction_by_ensemble.csv', index=False)

Save model

In [50]:
import pickle

# Save the trained model to a file
with open('ensemble_model.pkl', 'wb') as f:
    pickle.dump(voting_soft, f)